In [1]:
pip install pdf2image

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pdf2image import convert_from_path

In [3]:
import base64, os, gc, re
import pandas as pd
from pdf2image import convert_from_path
from io import BytesIO
from PIL import Image
from openai import OpenAI
from tqdm import tqdm
import openai

In [ ]:
# Cài đặt OpenAI Client
client = openai.OpenAI(
    api_key="",  # Thay API key
)

In [5]:
# --- CONVERT PDF PAGES TO IMAGES IN RAM ---
def convert_pdf_to_images_in_memory(pdf_path, dpi=200):
    return convert_from_path(pdf_path, dpi=dpi)

In [6]:
# --- CONVERT PIL IMAGE TO BASE64 ---
def image_to_base64_pil(pil_image):
    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

In [7]:
# --- BUILD EXAMPLE FOR GPT FROM PDF IN MEMORY ---
def create_image_content_from_pdf(pdf_path):
    images = convert_pdf_to_images_in_memory(pdf_path)
    image_contents = []
    for img in images:
        base64_img = image_to_base64_pil(img)
        image_contents.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{base64_img}"}
        })
    return image_contents

In [8]:
# --- PARSE GPT RESULT ---
def parse_sentiment_string(sentiment_str):
    pattern = r"#([\w\s&\-]+)\s(-?\d+\.?\d*)"
    return {match[0].strip(): float(match[1]) for match in re.findall(pattern, sentiment_str)}

In [9]:
init_system_prompt = f'''Bạn là trợ lý hữu ích giúp tạo các cặp 'khía cạnh' - 'cảm xúc' từ một 'văn bản chính thức', 'văn bản pháp lý', 'nghị quyết', 'báo cáo chính thức' của công ty do công ty công bố. Dưới đây là các yêu cầu khi trích xuất khía cạnh và cảm xúc:
1. Chúng tôi đã chuyển file pdf scan thành danh sách các ảnh. Bạn cần dùng khả năng xử lý ảnh (vision) của mình để “nhìn”, “đọc” và hiểu rõ nội dung trong các ảnh PNG để chấm điểm các khía cạnh cảm xúc.
2. Các khía cạnh được trích xuất bao gồm: Reputation (Danh tiếng của công ty); Company Communication (Truyền thông của công ty); Appointment (Bổ nhiệm); Financial (Tài chính); Regulatory (Cơ quan quản lí, chính sách); Sales (Bán hàng); M&A (Merge & Acquisition): Sáp nhập; Legal (Luật, tính hợp pháp); Dividend Policy (Chính sách cổ tức); Risks (rủi ro - ví dụ operation risk, credit risk - rủi ro tín dụng, reputation risk); Rumors (tin đồn); Strategy (chiến lược); Options (quyền chọn); IPO (initial public offering): lên sàn, niêm yết; Signal (tín hiệu để mua bán cổ phiếu); Coverage (báo cáo khuyến nghị buy sell hold của analysts); Fundamentals (các chỉ số trong phân tích cơ bản như P/E, P/B, Liabilites to Asset ratio); Insider Activity (các hoạt động nội bộ của công ty); Price Action (biến động giá); Buyside (các quĩ đầu tư, công ty nhỏ) ngược với sellside IB (Investment Banks); Technical Analysis (Phân tích kĩ thuật); Trade (thương mại, xuất nhập khẩu); Central Banks (Ngân hàng Trung ương, ở VN: Ngân hàng nhà nước, SBV); Currency (tiền tệ, tỉ giá); Conditions (điều kiện); Market (thị trường); Volatility (độ biến động, rủi ro); Investor Sentiment (Tâm lý chung của nhà đầu tư) lạc quan, lo lắng, chờ đợi...; Retail Investor Behavior (Hành vi của nhà đầu tư cá nhân) FOMO, hoảng loạn, giữ tiền mặt…; Speculation (Đầu cơ, hành vi đầu tư theo tin đồn hoặc kỳ vọng ngắn hạn); Domestic Institutional Behavior (Hành vi của tổ chức trong nước: tự doanh, bảo hiểm, quỹ trong nước) mua ròng, bán ròng, giảm tỷ trọng, rút khỏi cổ phiếu, các giao dịch thỏa thuận lớn giữa các tổ chức nội bộ; Foreign Institutional Behavior (Hành vi của nhà đầu tư tổ chức nước ngoài: ETF, quỹ ngoại, nhà đầu tư nước ngoài) mua ròng, bán ròng, tin tức đưa cổ phiếu vào, ra danh mục ETF, cảnh báo rủi ro, hạ mức xếp hạng cổ phiếu từ các tổ chức nước ngoài; Black Swan Event (Sự kiện thiên nga đen: hiếm gặp, khó dự đoán, gây ảnh hưởng nghiêm trọng như khủng hoảng, chiến tranh, dịch bệnh...).
3. Ảnh hưởng nội dung đến các khía cạnh tương ứng. Với mỗi chỉ số ảnh hưởng nên là số thực trong khoảng từ -1 (thật sự tiêu cực) đến 1 (thật sự tích cực)
4. Đảm bảo rằng các khía cạnh mà bạn trích xuất liên quan đến vấn đề của văn bản trên. Và khía cạnh nào bạn phân vân hãy xem không ảnh hưởng thì đánh là 0.
5. Chú ý là dựa vào ảnh hưởng của nội dung đến các khía cạnh. Khi cho tôi kết quả hãy theo format: có kí tự '#' trước khía cạnh, phía sau khía cạnh là con số ảnh hưởng nội dung, ví dụ về format '#khía cạnh1 1#khía cạnh2 -1#khía cạnh3 0'. Tuân thủ theo format của tôi, không làm khác.'''

In [10]:
query = "Dựa vào các hình ảnh sau, hãy đánh giá ảnh hưởng của nội dung trong ảnh đến từng khía cạnh theo yêu cầu đã mô tả."

In [11]:

init_assistant = """#Reputation 0.1#Company Communication 0.3#Appointment 0.0#Financial 0.5#Regulatory 0.0#Sales 0.0#M&A 0.0#Legal 0.0#Dividend Policy 0.0#Risks -0.1#Rumors 0.0#Strategy 0.0#Options 0.0#IPO 0.0#Signal 0.1#Coverage 0.0#Fundamentals 0.4#Insider Activity 0.0#Price Action 0.0#Buyside 0.0#Technical Analysis 0.0#Trade 0.0#Central Banks 0.0#Currency 0.0#Conditions 0.1#Market 0.0#Volatility 0.1#Investor Sentiment 0.2#Retail Investor Behavior 0.0#Speculation 0.0#Domestic Institutional Behavior 0.0#Foreign Institutional Behavior 0.0#Black Swan Event 0.0"""

In [12]:
# --- GỬI GPT ---
def analyze_pdf_scan_in_memory(pdf_path, init_user_prompt):
    image_contents = create_image_content_from_pdf(pdf_path)
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": init_system_prompt},
                {"role": "user", "content": [{"type": "text", "text": query}] + init_user_prompt},
                {"role": "assistant", "content": init_assistant},
                {"role": "user", "content": [{"type": "text", "text": query}] + image_contents},
            ],
            max_tokens=2048,
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"❌ GPT lỗi: {e}")
        return None
    finally:
        del image_contents
        gc.collect()

In [ ]:
pdf_path = r"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\official_statement_pdf\ACB\acb-thong-bao-giao-dich-co-phieu-cua-to-chuc-co-lien-quan-den-nguoi-noi-bo_0.pdf"
pdf_path_init_user_prompt = r"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\example_pdf_score\acb-giai-trinh-chenh-lech-lnst-quy-1-2023-so-voi-cung-ky-nam-truoc-0.pdf"
init_user_prompt = create_image_content_from_pdf(pdf_path_init_user_prompt)
sentiment_analysis_result = analyze_pdf_scan_in_memory(pdf_path, init_user_prompt)
print(sentiment_analysis_result)

#Reputation 0.2#Company Communication 0.3#Appointment 0.0#Financial 0.5#Regulatory 0.1#Sales 0.0#M&A 0.0#Legal 0.0#Dividend Policy 0.4#Risks 0.0#Rumors 0.0#Strategy 0.0#Options 0.0#IPO 0.0#Signal 0.1#Coverage 0.0#Fundamentals 0.3#Insider Activity 0.0#Price Action 0.0#Buyside 0.0#Technical Analysis 0.0#Trade 0.0#Central Banks 0.0#Currency 0.0#Conditions 0.0#Market 0.0#Volatility 0.0#Investor Sentiment 0.2#Retail Investor Behavior 0.0#Speculation 0.0#Domestic Institutional Behavior 0.0#Foreign Institutional Behavior 0.0#Black Swan Event 0.0


In [21]:
pdf_path = r"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\official_statement_pdf\ACB\acb-bao-cao-ket-qua-giao-dich-co-phieu-cua-nguoi-co-lien-quan-den-nguoi-noi-bo-nguyen-hong-nga.pdf"
pdf_path_init_user_prompt = r"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\example_pdf_score\acb-giai-trinh-chenh-lech-lnst-quy-1-2023-so-voi-cung-ky-nam-truoc-0.pdf"
init_user_prompt = create_image_content_from_pdf(pdf_path_init_user_prompt)
sentiment_analysis_result = analyze_pdf_scan_in_memory(pdf_path, init_user_prompt)
print(sentiment_analysis_result)

#Reputation 0.0#Company Communication 0.2#Appointment 0.0#Financial 0.0#Regulatory 0.0#Sales 0.0#M&A 0.0#Legal 0.0#Dividend Policy 0.0#Risks 0.0#Rumors 0.0#Strategy 0.0#Options 0.3#IPO 0.0#Signal 0.1#Coverage 0.0#Fundamentals 0.0#Insider Activity 0.4#Price Action 0.0#Buyside 0.0#Technical Analysis 0.0#Trade 0.0#Central Banks 0.0#Currency 0.0#Conditions 0.0#Market 0.0#Volatility 0.0#Investor Sentiment 0.0#Retail Investor Behavior 0.0#Speculation 0.0#Domestic Institutional Behavior 0.0#Foreign Institutional Behavior 0.0#Black Swan Event 0.0


In [13]:
# --- ANALYZE TỪNG FILE PDF TRONG EXCEL ---
def analyze_official_statements_for_stock(ticker):
    # folder_excel_input = rf"D:\...{ticker}"
    # folder_pdf_input = rf"D:\...{ticker}"
    # folder_excel_output = rf"D:\...{ticker}"
    folder_excel_input = rf"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\{ticker}"
    folder_pdf_input = rf"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\official_statement_pdf\{ticker}"
    folder_excel_output = rf"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\train_data"

    excel_path = os.path.join(folder_excel_input, f"{ticker}_filtered_news.xlsx")
    print(excel_path)
    df = pd.read_excel(excel_path)

    sentiment_columns = [
        'Reputation', 'Company Communication', 'Appointment', 'Financial', 'Regulatory',
        'Sales', 'M&A', 'Legal', 'Dividend Policy', 'Risks', 'Rumors', 'Strategy',
        'Options', 'IPO', 'Signal', 'Coverage', 'Fundamentals', 'Insider Activity',
        'Price Action', 'Buyside', 'Technical Analysis', 'Trade', 'Central Banks',
        'Currency', 'Conditions', 'Market', 'Volatility', 'Investor Sentiment', 'Retail Investor Behavior',
        'Speculation', 'Domestic Institutional Behavior', 'Foreign Institutional Behavior', 'Black Swan Event',
    ]
    for col in sentiment_columns:
        df[col] = None

    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            pdf_name = row.get("official_statement_pdf_name", "")
            if not isinstance(pdf_name, str) or not pdf_name.endswith(".pdf"):
                continue
            pdf_path = os.path.join(folder_pdf_input, pdf_name)
            if not os.path.exists(pdf_path):
                print(f"❌ Mất file: {pdf_name}")
                continue
            
            pdf_path_init_user_prompt = r"D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\example_pdf_score\acb-giai-trinh-chenh-lech-lnst-quy-1-2023-so-voi-cung-ky-nam-truoc-0.pdf"
            init_user_prompt = create_image_content_from_pdf(pdf_path_init_user_prompt)
            sentiment_str = analyze_pdf_scan_in_memory(pdf_path, init_user_prompt)
            if sentiment_str is None:
                continue

            sentiment_data = parse_sentiment_string(sentiment_str)
            for col in sentiment_columns:
                if col in sentiment_data:
                    df.at[i, col] = sentiment_data[col]

        except Exception as e:
            print(f"❌ Lỗi ở {pdf_name}: {e}")
            continue

    os.makedirs(folder_excel_output, exist_ok=True)
    output_path = os.path.join(folder_excel_output, f"{ticker}.xlsx")
    df.to_excel(output_path, index=False)
    print(f"✅ Đã lưu tại: {output_path}")

In [24]:
analyze_official_statements_for_stock("ACB")

D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\test_data\ACB\ACB_filtered_news.xlsx


100%|██████████| 12/12 [03:26<00:00, 17.20s/it]

✅ Đã lưu tại: D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\test_data\ACB.xlsx


In [14]:
def analyze_official_statements_for_vn30(tickers):
    for ticker in tickers:
        try:
            print(f"Đang xử lý cổ phiếu {ticker} test")
            analyze_official_statements_for_stock(ticker)
            print(f"Đã xử lý xong cổ phiếu {ticker} test")
        except Exception as e:
            print(f"Loi xu ly co phieu {ticker} test {e}")
            continue

In [ ]:
tickers = ['ACB','BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
analyze_official_statements_for_vn30(tickers)

Đang xử lý cổ phiếu SSI test
D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\SSI\SSI_filtered_news.xlsx


100%|██████████| 141/141 [40:26<00:00, 17.21s/it]


✅ Đã lưu tại: D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\train_data\SSI.xlsx
Đã xử lý xong cổ phiếu SSI test
Đang xử lý cổ phiếu STB test
D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\STB\STB_filtered_news.xlsx


100%|██████████| 211/211 [50:38<00:00, 14.40s/it]


✅ Đã lưu tại: D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\train_data\STB.xlsx
Đã xử lý xong cổ phiếu STB test
Đang xử lý cổ phiếu TCB test
D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\TCB\TCB_filtered_news.xlsx


...

✅ Đã lưu tại: D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\train_data\VNM.xlsx
Đã xử lý xong cổ phiếu VNM test
Đang xử lý cổ phiếu VPB test
D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\VPB\VPB_filtered_news.xlsx


100%|██████████| 227/227 [57:01<00:00, 15.07s/it]


✅ Đã lưu tại: D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\train_data\VPB.xlsx
Đã xử lý xong cổ phiếu VPB test
Đang xử lý cổ phiếu VRE test
D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\VRE\VRE_filtered_news.xlsx


100%|██████████| 128/128 [29:25<00:00, 13.79s/it]

✅ Đã lưu tại: D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\sentiment_score_data\official_statement\v2\train_data\VRE.xlsx
Đã xử lý xong cổ phiếu VRE test
